In [1]:
import tensorflow as tf

In [2]:
import unicodedata
import re
import numpy as np
import os
import io
import time

In [6]:
import pandas as pd

In [142]:
from sklearn.model_selection import train_test_split

In [168]:
from sklearn.utils import shuffle

In [70]:
path = 'data/'
friends = 'friends.csv'
bigbang = 'bigbang.csv'

In [79]:
friends_df = pd.read_csv(path+friends)
bigbang_df = pd.read_csv(path+bigbang)

In [80]:
friends_df[friends_df['dialogue'].isna()]

,episodes,speakers,dialogue


In [81]:
bigbang_df[bigbang_df['dialogue'].isna()].index

Int64Index([15625, 19508, 30537, 32482, 36957, 38247], dtype='int64')

In [82]:
df.head()

,episodes,speakers,dialogue,speaker_id
0,31,joey,I'm tellin' ya that girl totally winked at me.,1
1,31,other,"Did not, she did not wink at you....",14
2,31,chandler,Huh.,3
3,31,ross,I have to say Tupolo Honey by Van Morrison.,5
4,31,rachel,Nooo Way! The most romantic song ever is The ...,2


In [25]:
len(friends_df)+len(bigbang_df)

112706

In [83]:
df = pd.concat([friends_df, bigbang_df], ignore_index=True, sort=False)
len(df)

112706

In [84]:
df.reset_index(drop=True, inplace=True)

In [28]:
df.head()

,episodes,speakers,dialogue
0,31,joey,I'm tellin' ya that girl totally winked at me.
1,31,other,"Did not, she did not wink at you...."
2,31,chandler,Huh.
3,31,ross,I have to say Tupolo Honey by Van Morrison.
4,31,rachel,Nooo Way! The most romantic song ever is The ...


In [85]:
main_c = ['joey','rachel','chandler','monica','ross','phoebe','leonard','sheldon','penny','howard','raj','amy','bernadette','other']

In [33]:
speakers_ind=dict()
for ind, c in enumerate(main_c,1):
    speakers_ind[c] = ind
speakers_ind

{'amy': 12,
 'bernadette': 13,
 'chandler': 3,
 'howard': 10,
 'joey': 1,
 'leonard': 7,
 'monica': 4,
 'other': 14,
 'penny': 9,
 'phoebe': 6,
 'rachel': 2,
 'raj': 11,
 'ross': 5,
 'sheldon': 8}

In [86]:
df['speaker_id'] = df['speakers'].apply(lambda x: speakers_ind[x])
df.head()

,episodes,speakers,dialogue,speaker_id
0,31,joey,I'm tellin' ya that girl totally winked at me.,1
1,31,other,"Did not, she did not wink at you....",14
2,31,chandler,Huh.,3
3,31,ross,I have to say Tupolo Honey by Van Morrison.,5
4,31,rachel,Nooo Way! The most romantic song ever is The ...,2


In [87]:
speakerid_list = list(df['speaker_id'])
speakerid_list[:3]

[1, 14, 3]

In [88]:
speakers = list(df['speakers'])
dialogues = list(df['dialogue'])

In [89]:
episodes = list(df['episodes'])

In [110]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    if isinstance(w,float):
        w = '<SOS> <EOS>'
        return w
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<SOS> ' + w + ' <EOS>'
    return w

In [90]:
x = 12
isinstance(x, int)

True

In [39]:
print(preprocess_sentence(dialogues[0]))
print(preprocess_sentence(dialogues[0]).encode('utf-8'))

<SOS> i m tellin ya that girl totally winked at me . <EOS>
b'<SOS> i m tellin ya that girl totally winked at me . <EOS>'


In [92]:
data_size = len(df)

In [93]:
df[df['dialogue'].isna()]

,episodes,speakers,dialogue,speaker_id
77158,102,sheldon,NaN,8
81041,1019,other,NaN,14
92070,219,penny,NaN,9
94015,35,sheldon,NaN,8
98490,12,sheldon,NaN,8
99780,17,other,NaN,14


#### pack the batch of sequences of variable length
solution maybe: (not sure)<br>
tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')<br>
train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)<br>

In [139]:
def max_length(tensor):
    return max(len(t[0]) for t in tensor)

In [122]:
def tokenize(sentence,num_samples):
    sent_list = list()
    for k in range(0,num_samples):
        sent_list.append(preprocess_sentence(dialogues[k]))
    sent_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='')
    sent_tokenizer.fit_on_texts(sent_list)

    tensor = sent_tokenizer.texts_to_sequences(sent_list)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                     padding='post',
                                                     value=0)

    return tensor, sent_tokenizer,sent_list

In [123]:
tensor, tokenizer,sent_list = tokenize(dialogues,data_size)

In [162]:
count = 0
for key,val in tokenizer.word_index.items():
    count += 1
    print ("{0} ----> {1}".format(key, val))
    if count==3:
        break

electrical ----> 8652
vegetarianism ----> 18815
stwippers ----> 25038


In [164]:
# build up a dictionary index:word
index2word = {v: k for k, v in tokenizer.word_index.items()}

In [165]:
count = 0
for key,val in index2word.items():
    count += 1
    print ("{0} ----> {1}".format(key, val))
    if count==3:
        break

1 ----> .
2 ----> <sos>
3 ----> <eos>


In [124]:
tensor[0]

array([    2,     5,    22,  4010,   263,    14,   229,   402, 11626,
          63,    24,     1,     3,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [125]:
sent_list[0]

'<SOS> i m tellin ya that girl totally winked at me . <EOS>'

In [127]:
def create_dataset(tensor,num_samples):
    dialogues_list = list()
    response_list = list()
    for k in range(0,num_samples,2):
        if(k+1 >= num_samples):
            break
        if episodes[k]==episodes[k+1]:
            dialogue = tensor[k]
#             pdb.set_trace()
            response = tensor[k+1]
            speaker = speakerid_list[k]
            addressee = speakerid_list[k+1]
            dialogues_list.append([dialogue,speaker])
            response_list.append([response,addressee])
#     print(dialogues_list)
#     print(response_list)
    return dialogues_list,response_list    

In [128]:
d_tensor,r_tensor = create_dataset(tensor,data_size)

In [129]:
len(d)

56123

In [140]:
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(r_tensor), max_length(d_tensor)
print(max_length_targ)
print(max_length_inp)

262
262


In [184]:
# arr1 = [1,2,3,4]
# arr2 = [11,22,33,44]
# shuffle(arr1,arr2)

[[3, 1, 2, 4], [33, 11, 22, 44]]

In [185]:
# shuffle
tensor = shuffle(d_tensor,r_tensor)

In [186]:
len(tensor)

2

In [187]:
d_tensor = tensor[0]
r_tensor = tensor[1]

In [188]:
# Creating training and validation sets using an 80-20 split
d_tensor_train, d_tensor_val, r_tensor_train, r_tensor_val = train_test_split(d_tensor, r_tensor, test_size=0.2)

# Show length
print(len(d_tensor_train), len(r_tensor_train), len(d_tensor_val), len(r_tensor_val))

44898 44898 11225 11225


In [191]:
dia_train = [t[0] for t in d_tensor_train]
dia_val = [t[0] for t in d_tensor_val]
sid_train = [t[1] for t in d_tensor_train]
sid_val = [t[1] for t in d_tensor_val]
res_train = [t[0] for t in r_tensor_train]
res_val = [t[0] for t in r_tensor_val]
aid_train = [t[1] for t in r_tensor_train]
aid_val = [t[1] for t in r_tensor_val]

In [194]:
BUFFER_SIZE = len(d_tensor_train)
BATCH_SIZE = 128
steps_per_epoch = len(d_tensor_train)//BATCH_SIZE
HIDDEN_SIZE = 1000
NUM_LAYER = 4
DROP_OUT = 0.2
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 512
speakerNum = len(main_c)

In [192]:
# create tf.dataset
dataset = tf.data.Dataset.from_tensor_slices((dia_train,res_train, sid_train,aid_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)

   ## Encoder and Decoder

In [166]:
class Encoder(tf.keras.Model):
    def __init__(self, hidden_size, vocab_size,embedding_dim, num_layers=1, batch_size=1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.input_size = embedding_dim
            
        self.lstm = tf.keras.layers.LSTM(self.hidden_size,
                                       return_sequences=True,
                                       return_state=True,
                                       dropout=DROP_OUT,
                                       recurrent_initializer='glorot_uniform')

    def call(self, d, hidden):
        d = self.embedding(d)
        # four layer train, 4 lstm
        for k in range(num_layer):
            output, hidden = self.lstm(d, initial_state = hidden)
            d = output
        return output, hidden_state

    def initialize_hidden_state(self,batch_size=0):
        if batch_size == 0: batch_size =self.batch_size
        return tf.zeros((self.batch_size, self.hidden_size))

In [153]:
class Attention_Feed(tf.keras.Model):
    def __init__(self, hidden_size):
        super(Attention_Feed, self).__init__()
        self.W1 = tf.keras.layers.Dense(hidden_size)
        self.W2 = tf.keras.layers.Dense(hidden_size)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))
#         print("score.size():{0}".format(score.shape))
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
#         print("values.size():{0}".format(values.shape))
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
#         print("context_vector.size():{0}".format(context_vector.shape))
        return context_vector, attention_weights

In [156]:
def combine_user_vector(i_tensor,j_tensor):
    size = s_tensor.shape[-1]
    W1 = tf.keras.layers.Dense(size)
    W2 = tf.keras.layers.Dense(size)
    V_ij = tf.nn.tanh(W1(i_tensor) + W2(j_tensor))
    return V_ij

In [196]:
class Decoder(tf.keras.Model):
    def __init__(self, hidden_size, vocab_size,embedding_dim, num_layers=1):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.speaker_embedding = tf.keras.layers.Embedding(speakerNum, embedding_dim)
        self.input_size = embedding_dim
        self.output_size = vocab_size #vocabulary size           
        self.lstm = tf.keras.layers.LSTM(self.hidden_size,
                                       return_sequences=True,
                                       return_state=True,
                                       dropout = DORP_OUT,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.output_size)
        
        # attention feed on context
        self.attention = Attention_Feed(self.hidden_size)

    def call(self, x, hidden, enc_output,speaker_id,addressee_id=None):
#         batch_size = x.size()[1]
        
        context_vector, attention_weights = self.attention(hidden, enc_output)
        features = self.embedding(x)
        # personas
        speaker = self.speaker_embedding(speaker_id)
        if addressee_id is not None:
            addressee = self.speaker_embedding(addressee_id)
            v_ij = combine_user_vector(speaker,addressee)
            features = tf.concat([features, v_ij], axis=-1)
        else:
            features = tf.concat([features, speaker], axis=-1)
#         max_length = enc_output.size(0)  
        r = tf.concat([tf.expand_dims(context_vector, 1), features], axis=-1)
        
        # passing the concatenated vector to the 4-layer LSTM
        for k in range(num_layers):
            output, state = self.lstm(r)
            r = output

        # Removes dimensions of size 1 from the shape of a tensor.
        # output shape: (batch_size, 1, hidden_size) --> (batch_size *1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
#         output = output.squeeze(0)

        # output shape == （batch_size, hidden_size)
        output = tf.nn.log_softmax(self.fc(output),axis=1)

        return output, state,attention_weights

In [ ]:
# from tf tutorial
# loss = 0

# with tf.GradientTape() as tape:
#     enc_output, enc_hidden = encoder(inp, enc_hidden)

#     dec_hidden = enc_hidden

#     dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

#     # Teacher forcing - feeding the target as the next input
#     for t in range(1, targ.shape[1]):
#         # passing enc_output to the decoder
#         predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

#         loss += loss_function(targ[:, t], predictions)

#         # using teacher forcing
#         dec_input = tf.expand_dims(targ[:, t], 1)

#     batch_loss = (loss / int(targ.shape[1]))

#     variables = encoder.trainable_variables + decoder.trainable_variables

#     gradients = tape.gradient(loss, variables)

#     optimizer.apply_gradients(zip(gradients, variables))

#     return batch_loss

In [167]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [197]:
class Train(object):
    def __init__(self,encoder,decoder,optimizer,index2word,num_layers=1):
        self.encoder = encoder
        self.decoder = decoder
        self.index2word = index2word
        self.num_layers = num_layers
        self.optimizer = optimizer
    
    def train_step(self,inp, targ, speaker_id,addressee_id=None):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output,enc_hidden = encoder(inp,enc_hidden)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([tokenizer.word_index['<sos>']]*BATCH_SIZE,1)
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                if addressee_id is not None:
                    predictions, dec_hidden, _ = decoder(
                        dec_input, dec_hidden,enc_output,speaker_id,addressee_id)
                else:
                    predictions, dec_hidden, _ = decoder(
                        dec_input, dec_hidden,enc_output,speaker_id)
                    
                loss += loss_function(targ[:, t], predictions)

                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
            
            batch_loss = (loss / int(targ.shape[1]))
            
            variables = encoder.trainable_variables + decoder.trainable_variables

            gradients = tape.gradient(loss, variables)

            self.optimizer.apply_gradients(zip(gradients, variables))
            

In [198]:
EPOCHS = 10

In [ ]:
# need to modify
def run_iter(epochs,isAddressee):
    for e in range(epochs):
        start = time.time()

        enc_hidden = encoder.initialize_hidden_state()
        total_loss = 0

        if isAddressee==True:
            for (batch, (inp, targ,sid,aid)) in enumerate(dataset.take(steps_per_epoch)):
            batch_loss = train_step(inp, targ, enc_hidden,sid,aid)
            total_loss += batch_loss

            if batch % 100 == 0:
                print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                             batch,
                                                             batch_loss.numpy()))
        else:
            for (batch, (inp, targ,_,sid)) in enumerate(dataset.take(steps_per_epoch)):
                batch_loss = train_step(inp, targ, enc_hidden,sid)
                total_loss += batch_loss

                if batch % 100 == 0:
                    print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                                 batch,
                                                                 batch_loss.numpy()))
        # saving (checkpoint) the model every 2 epochs
        if (epoch + 1) % 2 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)

        print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                          total_loss / steps_per_epoch))
        print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))